# Uniswap V2
For this strategy, we will provide the liquidity just as in Uniswap v2. We will provide liquidity in the interval [minTick, maxTick].

Import code dependencies

In [13]:
from datetime import date, datetime
import pandas as pd

from demeter import TokenInfo, Actuator, Strategy, RowData, ChainType, MarketInfo, AtTimeTrigger
from demeter.metrics import performance_metrics
from demeter.uniswap import UniV3Pool, UniLpMarket

Set pandas output format

In [14]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

Custom Uniswap V2 strategy with add liquidity from tick -887272(minTick) to 887272(maxTick)

In [15]:
class UniswapV2Strategy(Strategy):
    def initialize(self):
        """
        Initialize function will be called right before a backtest start.
        You can do various things here, e.g. register a trigger, or add a simple moving average line.
        """
        new_trigger = AtTimeTrigger(  # define a new trigger
            time=datetime(2023, 8, 15, 12, 0, 0), do=self.work  # we will make the action happen at 12:00 20/8/22
        )  # This is a callback function, defines what to do at this time.
        self.triggers.append(new_trigger)  # Register our trigger

    def work(self, row_data: RowData):
        """
        When time is up, work function will be called.
        """
        lp_market: UniLpMarket = self.markets[market_key]  # pick our market.
        lp_market.add_liquidity_by_tick(-887272, 887272)  # add liquidity
        pass

Main logic to run Actuator, init two token and market with key "market1"

In [16]:
# Declare a token, and it's name will be used as unit of amounts.
usdc = TokenInfo(name="usdc", decimal=6)  # declare token usdc
eth = TokenInfo(name="eth", decimal=18)  # declare token eth
# Declare an Uniswap V3 pool. We will set the parameters according to the real pool on chain.
pool = UniV3Pool(token0=usdc, token1=eth, fee=0.05, quote_token=usdc)

# Declare a market key, which will be used to find the corresponding market in broker
market_key = MarketInfo("market1")
# Declare the market,
market = UniLpMarket(market_key, pool)  # uni_market:UniLpMarket, positions: 0, total liquidity: 0
# load data for market. those data is prepared by download tool
market.data_path = "../data"  # set data path
market.load_data(
    chain=ChainType.polygon.name,  # load data
    contract_addr="0x45dda9cb7c25131df268515131f647d726f50608",
    start_date=date(2023, 8, 15),
    end_date=date(2023, 8, 15),
)

# Declare the Actuator, which controls the whole process
actuator = Actuator()  # declare actuator, Demeter Actuator (broker:assets: ; markets: )
# add market to broker
actuator.broker.add_market(market)
# Initial some fund to broker.
actuator.broker.set_balance(usdc, 10000)
actuator.broker.set_balance(eth, 10)
# Set strategy to actuator
actuator.strategy = UniswapV2Strategy()  # set strategy to actuator
# Set price. Those price will be used in all markets.
# Usually, you will have to find the price list from outer source.
# Luckily, uniswap pool data contains price information. So UniLpMarket provides a function to retrieve price list.
actuator.set_price(market.get_price_from_data())
# run test, If you use default parameter, final fund status will be printed in console.

# actuator.run()

2024-07-22 16:21:34,809 - INFO - start load files from 2023-08-15 to 2023-08-15...
2024-07-22 16:21:34,824 - INFO - load file complete, preparing...
2024-07-22 16:21:34,891 - INFO - data has been prepared


Run actuator with evaluators and save result to files

In [17]:
actuator.run()
print({k: round(v, 5) for k, v in performance_metrics(
    actuator.account_status_df["net_value"], benchmark=actuator.account_status_df["price"]["ETH"]
).items()})

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2024-07-22 16:21:34,910 - INFO - init strategy...
2024-07-22 16:21:34,915 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1440/1440 [00:00<00:00, 1566.44it/s]
2024-07-22 16:21:35,841 - INFO - main loop finished
2024-07-22 16:21:35,900 - INFO - Print actuator summary
2024-07-22 16:21:35,953 - INFO - Backtesting finished, execute time 1.0433599948883057s
2024-07-22 16:21:35,989 - INFO - files have saved to ./result\backtest-20240722-162135.account.csv,./result\backtest-20240722-162135.action.json,./result\backtest-20240722-162135.action.pkl


Final account status                              
Token balance in broker       
USDC      :0                        ETH       :4.5592684                
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee(%)    :0.0500                   quote token:USDC                     
positions 
   lower_tick  upper_tick     pending0        pending1        liquidity
0     -887272      887272  0.096610981  0.000065859322  233253759509000

Account balance history                           
l1                  net_value tokens             market1                                                                                          price     
l2                              USDC       ETH net_value base_uncollected quote_uncollected base_in_position quote_in_position position_count       ETH USDC
2023-08-15 00:00:00 28436.944  10000        10         0                0                 0                0                 0

['./result\\backtest-20240722-162135.account.csv',
 './result\\backtest-20240722-162135.action.json',
 './result\\backtest-20240722-162135.action.pkl']